# Create feature vectors

Since the annottaions are based on 5s (5\*30fps), 
features are based on 64 frames (~2.5s)

two things to be done to match the labels invistigation:
- keep as is and match with the 2.5 lables 
- Fuse the feature vector to create a 5s equevelent 
- (1) concatenate: reshape the feature array to one long vector (5s) 
- (2) avergae the feature victors 


### imports & variables

In [12]:
import sys
import numpy as np
import os
import pandas as pd

lables_path = './labels/'
all_folders_path = './features/'

### help functions

In [19]:
def person_new_features(this_file,feature_folder, featureType, fusionType, divide):
    currData = np.load(os.path.join(feature_folder,this_file))
    reshape_span =2
    t = currData.shape
    cutoff = int(currData.shape[0]/2)*2 #<-- remove any extra 2.5 segment 
    currData=currData[:cutoff]
    
    if fusionType == 'avg':
        currData = currData.transpose().reshape(-1,reshape_span).mean(1).reshape(currData.shape[1],-1).transpose()
    else:
        currData = currData.reshape((int(currData.shape[0]/2),int(currData.shape[1]*2)))
    
    
    #folder like: i3d_flow_features_avg_5
    new_feature_folder = '_'.join(['i3d',featureType,'features',fusionType,str(divide)])
    os.makedirs(os.path.join(all_folders_path,new_feature_folder), exist_ok=True)
    
    
    #print('{} in {} : {} -> {}'.format(new_feature_folder,this_file,t,currData.shape))
    
    np.save(os.path.join(all_folders_path,new_feature_folder,this_file), currData)
    
def create_new_features(feature_folder,featureType, fusionType, divide):
    
    onlyfiles = [f for f in os.listdir(feature_folder) if
                   os.path.isfile(os.path.join(feature_folder, f))]
    onlyfiles.sort()

    for this_file in onlyfiles:
        person_new_features(this_file,feature_folder, featureType, fusionType, divide)

### create the files

In [20]:
for featureType in ['rgb','flow']:
    for fusionType in ['conc','avg']:
        divide = 5
        print('Working on {}_{}_{}'.format(featureType, fusionType, divide))
        create_new_features(os.path.join(all_folders_path,'i3d_'+featureType+'_features'), 
                             featureType, fusionType, divide)

Working on rgb_conc_5
Working on rgb_avg_5
Working on flow_conc_5
Working on flow_avg_5
